In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sentence_transformers import SentenceTransformer
import pickle 

from IPython.display import clear_output
clear_output(wait=False)

In [2]:
anime_df = pd.read_csv("//Users/shitanshubhushan/Documents/Anime Recommendation/data/anime_with_synopsis.csv")
user_df = pd.read_csv("/Users/shitanshubhushan/Documents/Anime Recommendation/data/animelist.csv")

## Anime

### Drop Animes with no Synopsis

In [3]:
MAL_ID_todrop = set(anime_df[anime_df['sypnopsis'].isna()]['MAL_ID'].reset_index(drop=True))
anime_df = anime_df[~anime_df['MAL_ID'].isin(MAL_ID_todrop)]

### Embed all synopsis

In [4]:
embed = []
MAL_ID = []

model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
clear_output(wait=False)

for index, row in anime_df.iterrows():
    embed.append(model.encode(row['sypnopsis']))
    MAL_ID.append(row['MAL_ID'])

    clear_output(wait=False)

embed_df = pd.DataFrame({'Synopsis_embed': embed,
                         'MAL_ID':MAL_ID
                        })

anime_df = anime_df.merge(embed_df, left_on='MAL_ID', right_on='MAL_ID')

### Genre Embedding

In [5]:
genres = set()

for index, row in anime_df.iterrows():
    genres.update(row['Genres'].split(","))

genre_df = pd.DataFrame(0, index=np.arange(anime_df.shape[0]), columns=list(genres))

for index, row in anime_df.iterrows():
    anime_genres = row['Genres'].split(",")
    for genre in anime_genres:
        genre_df.loc[index, genre]=1

genre_df = genre_df.astype(str)
genre_embed_df = pd.DataFrame({'Genre_embed':genre_df.apply(lambda row: np.asarray(row.values.tolist()), axis=1),
                               'MAL_ID': MAL_ID})

anime_df = anime_df.merge(genre_embed_df, left_on='MAL_ID', right_on='MAL_ID')

### Score Embedding

In [6]:
anime_score_median = anime_df['Score'].replace('Unknown',0).astype(float).median()
anime_df['Score'] = anime_df['Score'].replace('Unknown',anime_score_median).astype(float)
anime_df['Score_embed'] = anime_df['Score']/10
anime_df.drop(columns=['Score','Genres','sypnopsis'],inplace=True)

### Cumulative Embedding

In [7]:
#anime_df['anime_embedding'] = anime_df.apply(lambda row: np.append(np.concatenate((row['Synopsis_embed'],
#                                                     row['Genre_embed'])),row['Score_embed']), axis=1)
anime_df['anime_embedding'] = anime_df['Synopsis_embed']
anime_df.drop(columns=['Score_embed','Genre_embed','Synopsis_embed'],inplace=True)

### Saving Stuff

In [8]:
with open('output/anime_df.pkl', 'wb') as f:
    pickle.dump(anime_df, f)

In [9]:
anime_dict = anime_df.set_index('Name')['MAL_ID'].to_dict()
with open('output/anime_name_dict.pkl', 'wb') as f:
    pickle.dump(anime_dict, f)

## User Grouping

In [10]:
user_df = user_df[(user_df['watching_status']==1) | (user_df['watching_status']==2) | (user_df['watching_status']==3)]
user_df = user_df[user_df['rating']>=7]
user_df.drop(columns=['watched_episodes','watching_status','rating'],inplace=True)
u = user_df.groupby("user_id")["anime_id"].agg(list).reset_index()

In [11]:
with open('output/user_df.pkl', 'wb') as f:
    pickle.dump(u, f)